# Технологии обработки больших данных
Занятие 1. Введение
1. Административная часть
2. Обработка текстовых файлов
3. Функциональная парадигма программирования

### 1. Административная часть
 Бально-рейтинговая система:  

Текущий контоль успеваемости:
* Выполнение домашних заданий - 30 баллов 
* Активность на занятих - 10 баллов

Промежуточная аттестация:
* Теоретическая часть - 30 баллов
* Практическая часть - 30 баллов 

За практическую часть возможно будет засчитан индивидуальный проект (если не будет курсовой работы).

### 2. Обработка текстовых файлов
Log generation  

Пусть у нас есть интернет-магазин, который посещают 10 посетителей в час. Итого за год имеем 87 600 посетителей (у wildberries - около 40 млн).  

Каждый посетитель совершает какие-то действия на сайте, которые записываются в log файл. Мы имитируем эти действия с помощью 100 случайных слов (из трех случайных букв).  

Наш лог файл получился небольшого размера и легко помещается в оперативную память одной машины. Но так бывает не всегда. Есть два пути решения проблемы.

<img src="img/scaling.png">

<img src="img/threads.png">

<img src="img/pipeline.jpg">

In [1]:
import random, string, datetime

In [2]:
letters = string.ascii_lowercase

LOG_SIZE = 10*24*365 # 87600

def get_random_word(length=3):
   return ''.join(random.choice(letters) for i in range(length))

def get_line(length=100):
  t = str(datetime.datetime.now())
  words = [get_random_word() for n in range(length)]
  return t + ' ' + ' '.join(w for w in words) + '\n'

lines = [get_line() for l in range(LOG_SIZE)]

with open('log.txt', 'w') as f:
  f.writelines(lines)

In [3]:
with open('log.txt') as f:
  lines = f.readlines()

print(f'Total lines - {len(lines)}. First 5:')
lines[:5]

Total lines - 87600. First 5:


['2022-02-16 17:47:42.154547 zjv gfu dam iro hoj wuu biv lez qbw lgv dkz wvj tzj zdu iro ouo ydo mhx hsw tyi dwi pbj pww uwg sui mdi yss dks sev pnr bqh kkm svh opr eor kxq uau hir gga wsu wuj nqx uif qmk urg ccw bgl vkv wap ffa mfb ejg uig yig pbs hpk jtp dfm fpb ift ndv iiz bsm qsp zvu eny gma ekn sat hgk bgz kkm qmu cpi gmw qde wxc nab uiw piu ati clz aze fie tqt utg swn mkv ioj ljx img rli saj rsc tqx yqy szr mky kbs nob\n',
 '2022-02-16 17:47:42.155723 xhx xpp hnu axu sji xhy seq lhi sfi qdg kqq ytp jbu olb doh omn xap clr xxq dwm dco zfp lnz eyj zsx stk dtd eae fno aio xpu vuq kfa dff vdu jiu nwi iuu xse iwv nrs bmn nrm kod gtb fpt oxb ejj qfk yua hdb eny tgh ldb ezb avy fzi zqv hhf dxm tfk uak mes gko cxy eye kmj slf pwl hls ygk hhn pon gnd uiu sgf atb wuv gei zzq omc wfh yzg puu gfu sbf hvp iay sdz hby zhp hnw nmj fxk xnd zqy tnp czi nut vft\n',
 '2022-02-16 17:47:42.156961 urw wwl mec ism fkg pub byj ihz vqr hjk ruy ebn uel wkq iaa box gge pdu zxo ubo xjh xmi zkl imz hup ymq c

Проведем простую аналитику: сколько раз встречается слово 'cat' и слово 'dog' в нашем файле.

In [10]:
%%time 

cat_count = 0
dog_count = 0

for line in lines:
  if 'cat' in line:
    cat_count += 1
  if 'dog' in line:
    dog_count += 1

print(f'cat - {cat_count}, dog - {dog_count}')

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11.2 µs
cat - 480, dog - 474


Немного усложним задачу: сколько раз встречается каждое слово? 

In [11]:
from collections import Counter

words = []

for line in lines:
  words += line.split(' ')[2:]

In [13]:
# 87600 * 100
len(words)

8760000

In [18]:
%%timeit
counts = Counter(words)

1.36 s ± 19.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Попробуем ускорить вычисления за счет разделения списка слов

In [20]:
%%time

LEN = len(words) // 2

words1 = words[:LEN]
words2 = words[LEN:]

counts1 = Counter(words1)
counts2 = Counter(words2)

counts3 = counts1 + counts2

CPU times: user 1.31 s, sys: 36.1 ms, total: 1.35 s
Wall time: 1.37 s


In [21]:
counts3 == counts

True

Или с помощью параллельных процессов

In [22]:
from multiprocessing import Process, Queue, cpu_count

q = Queue()

def count(q, words):
    c = Counter(words)
    q.put(c)
    
c = cpu_count()
print(f'CPU count - {c}')

CPU count - 4


In [25]:
%%timeit
p1 = Process(target=count, args=(q, words1))
p1.start()

ac1 = q.get()
p1.join()

878 ms ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit
p1 = Process(target=count, args=(q, words1))
p1.start()

p2 = Process(target=count, args=(q, words2))
p2.start()

ac1 = q.get()
ac2 = q.get()

p1.join()
p2.join()

async_count = ac1 + ac2

1.13 s ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
async_count == counts

True

### 3. Функциональная парадигма программирования
What is map reduce?  

Greeting programming language paradigm:
1. Imperative:
* Procedural (Fortran, Pascal, C ...)
* Object Oriented (Smalltalk, C++, Java ...)
* Parallel Processing (Ada, Occam, cuda ...)
2. Declarative:
* Logic (Prolog ...)
* Functional (Lisp, Scheme ...)
* Database (SQL ...) 

Большинство современных языков поддерживают несколько парадигм программирования.

#### High-order functions (Функции высшего порядка)
#### map

In [61]:
data = list(range(5))
data

[0, 1, 2, 3, 4]

In [53]:
%%time
def my_cube(x):
  return x*x*x

my_map = map(my_qube, data)

CPU times: user 10 µs, sys: 1e+03 ns, total: 11 µs
Wall time: 16.5 µs


In [49]:
%%time
my_list = list(my_map)

CPU times: user 31.6 ms, sys: 8.09 ms, total: 39.7 ms
Wall time: 51.8 ms


In [57]:
%%timeit
my_list = []
for d in data:
    my_list.append(my_cube(d))

13.2 ms ± 503 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [59]:
%%timeit
list(map(lambda x: x*x*x, data))

8.79 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### filter 

In [60]:
list(filter(lambda x: x%2==0, data))

[0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48,
 50,
 52,
 54,
 56,
 58,
 60,
 62,
 64,
 66,
 68,
 70,
 72,
 74,
 76,
 78,
 80,
 82,
 84,
 86,
 88,
 90,
 92,
 94,
 96,
 98,
 100,
 102,
 104,
 106,
 108,
 110,
 112,
 114,
 116,
 118,
 120,
 122,
 124,
 126,
 128,
 130,
 132,
 134,
 136,
 138,
 140,
 142,
 144,
 146,
 148,
 150,
 152,
 154,
 156,
 158,
 160,
 162,
 164,
 166,
 168,
 170,
 172,
 174,
 176,
 178,
 180,
 182,
 184,
 186,
 188,
 190,
 192,
 194,
 196,
 198,
 200,
 202,
 204,
 206,
 208,
 210,
 212,
 214,
 216,
 218,
 220,
 222,
 224,
 226,
 228,
 230,
 232,
 234,
 236,
 238,
 240,
 242,
 244,
 246,
 248,
 250,
 252,
 254,
 256,
 258,
 260,
 262,
 264,
 266,
 268,
 270,
 272,
 274,
 276,
 278,
 280,
 282,
 284,
 286,
 288,
 290,
 292,
 294,
 296,
 298,
 300,
 302,
 304,
 306,
 308,
 310,
 312,
 314,
 316,
 318,
 320,
 322,
 324,
 326,
 328,
 330,
 332,
 334,
 336,
 338,
 340,
 342,
 344,
 346,
 348,
 350,

#### reduce 

In [62]:
data

[0, 1, 2, 3, 4]

In [63]:
import functools as f

def my_sum(a, b):
  return a + b

f.reduce(my_sum, data)

10

In [64]:
f.reduce(lambda a, b: a + b, data)

10

# Домашнее задание

#### 1. Переведите все слова в лог-файле в uppercase, в решении используйте map 

In [ ]:
# expected
"""['2022-02-12 14:07:14.497548 ERK QJF REP DQL ZSC SHL BNB AAP OAU XML BKU PUD OYU LRZ KUF LYC XKH WVQ UVE LEW RVN HOD ZFJ SHF PAS YFC VTZ JPA GHU TJD HSS MPS NEO EYJ GNY VDD XLD XIK UMZ KBN WNE GYW MBX LNI TTA QCN BHF KUZ WCQ QBJ SHI YHR DLL MVH SPV BTG MTJ XLN AIY HKA BQO ZZH BBI XRH NEP CNK KUV RNE FNU SCZ KFV EHM WYI EWE OAP UWF IVJ HFL ODJ SOE RTE FXE OLY TOG VUX KYP RUK MCQ COO TBD XBI CUG TLB GJY GKQ TEO UZF SIR RGK KMP\n',
 '2022-02-12 14:07:14.498591 VIS BRA CHC GAM FYT VCP DWO VHT NEM LAA CXS YRC KOY ZTJ TZC MJI XPJ QBA XMT PJX QBE POY EUC MLH FPU CNX PSG QLN QMF APS URL GZZ CSU FYY IME LJG GGI LJA EDD TVG SJM JIZ EQM LRK GUN AQZ IGQ SBA LZU HUJ EQM HZJ KKJ JHE GJW ELG RXA RLH VBL YMR LDJ ISY VJX GVV UFX NYY LHG QYV XMT JET KRJ MNJ JKH MJL WVT RVW MOJ OES QXA ZJW DQV VNE ZEP UGV GXZ YCB FPS BZF ENH WSH PJD WED CLX WBN QGS BOP QFF ZWK CRQ XJR\n',
 '2022-02-12 14:07:14.499505 WXG DLB AKS IXN JEK QGD TUY KSU HDF CEY JEQ GLU MAW TEW YZO AVC IBD YSS HYL YBO DCF PNT UTB MZW ZVJ EYN YYW FLM TIL WYG WJR ETO XMD KRL OOE MEN HED QRJ JZC SFT RXN JNC PEG CWV TKO XRU YEP AQW ZZV VNU PYU THS PML ZEG XIK RYF DRS VFQ GLB FDZ SCT KEP GXP NKL XTE TRF LIX CDB ARM NBT GCV BNU AFX PPG PRV DYX AJN ZDJ QTU IRB QLI LNG RJR LFN KBD JTV IPX OIV YUS DZR MXG QHM CTI LLM IUT ZCK RCT QPN LQS OYO\n',
 '2022-02-12 14:07:14.500389 WOY WFE LUU BWK YBR FNU BRY CFR YWS TVA ABR TJI MWU THK NMJ HLB NFG ICR ITF DAY ZFB ZZI QHA VLA KZW WEW QYY IGI XLW VWB RYI HDH CTN MQE OYI ICD IEC CGL OIL GYR ZYI SGW DWU KQB CTR YIZ XHT EVM OOA UAF JUO NBK XMD VOM ZTQ GLO FTZ BPZ XNP FNH IQJ DAO CYN CDE MXS CEB VUV GEC BZI SRF XYH PWE UEZ QVA FKN WKA TJM VUI HSI EGS UTL WBT TUG JYC KOH NLO AYF QAZ FYM AFA WPH WKV DMA JDP NDW ROF SUQ QOK QZP PFZ\n',
 '2022-02-12 14:07:14.501292 ZWL QPO HOC WLB HRM DOM DIQ CKZ INA HBY DQR OMQ UWN NCW YIY GYB ZZZ HIC CHM LUY CIX MYD PPH NEK VUP IFH VRW XSX RMI RFH QLW WBT LDQ BXN YMA HXX INQ YZS NZI LCN AGU GNW MGB XYP WTM QMI QJC UOU RAC UAR NPD ELS CIN OMN QRJ KLU AIZ OVL GLF VIB QHF SRX CLH DGJ DQY CXK OQO IRC TKK PFZ PVE FBN NQT NDQ VMC ZGT KBI SOD QBB MQR LSL YMM MIQ HZF MGQ ZLM SZO YOL HWC WIY RAI NKK VAL VWM FPW PHL TZS DVU WKQ DNJ\n']"""

#### 2. Создайте новый список с логами, в которых будут только слова из white_list.txt, в решении используйте map и filter

In [ ]:
# expected 
"""['2022-02-1214:07:14.497548 sir',
 '2022-02-1214:07:14.498591 jet',
 '2022-02-1214:07:14.499505 peg arm',
 '2022-02-1214:07:14.500389 day tug',
 '2022-02-1214:07:14.501292 ']"""

#### 3. Посчитайте количество слов из white_list в файле log.txt

In [ ]:
# expected
"""Counter({'arm': 158,
         'tug': 154,
         'may': 175,
         'tan': 194,
         'cry': 184,
         'fin': 174... })"""